<center><h1 style="background-color: #C6F3CD; border-radius: 10px; color: #FFFFFF; padding: 5px;">
Genetic Algorithms

</h1><center/>

**Link to the article** :  

In [1]:
import numpy as np
import pandas as pd
np.random.seed(42)

## 1) Process / fitness functions

In [2]:
def process_quality(x):
    temp, pres, flow = x
    return (
        np.sin(temp/50) * 10 +
        np.cos(pres*2) * 5 +
        -0.01 * (flow - 60)**2 + 20
    )

def fitness(individual):
    return process_quality(individual)

## Optional if you want minimization instead

In [28]:
# Minimize cost 
def fitness(ind):
    return - process_cost(ind)

In [23]:
# Quality − λ·cost tradeoff
def fitness(ind):
    return process_quality(ind) - 0.5 * energy_cost(ind)

In [24]:
# Minimize energy
def fitness(ind):
    return - energy_cost(ind)

## 2) Hyperparameters & bounds

In [3]:
POP_SIZE = 6
N_GEN = 5
MUT_RATE = 0.3
LOW = np.array([0, 0, 0])
HIGH = np.array([300, 10, 100])

## 3) Initialize population

In [4]:
population = np.random.uniform(low=LOW, high=HIGH, size=(POP_SIZE, 3))

## 4) GA components

In [5]:
def evaluate_population(pop):
    return np.array([fitness(ind) for ind in pop])


def roulette_selection(pop, fit):
    # Shift fitness to avoid negatives
    shifted = fit - fit.min() + 1e-8
    probs = shifted / shifted.sum()
    idx = np.random.choice(len(pop), size=len(pop), p=probs, replace=True)
    return pop[idx]


def single_point_crossover(parents):
    children = []
    n = parents.shape[1]  # number of genes
    for i in range(0, len(parents), 2):
        p1 = parents[i]
        p2 = parents[(i+1) % len(parents)]
        k = np.random.randint(1, n)  # crossover point
        c1 = np.concatenate([p1[:k], p2[k:]])
        c2 = np.concatenate([p2[:k], p1[k:]])
        children.append(c1)
        children.append(c2)
    return np.array(children)


def mutate(children, rate=MUT_RATE, sigma=np.array([5.0, 0.2, 2.0])):
    mask = np.random.rand(*children.shape) < rate
    noise = np.random.normal(0, 1, size=children.shape) * sigma
    children = children + mask * noise
    return np.clip(children, LOW, HIGH)

## 5) Put everything in one place

In [6]:
best_scores = []
best_individuals = []

for gen in range(N_GEN):
    # 1. Evaluate
    fitness_scores = evaluate_population(population)

    # Track best
    best_idx = np.argmax(fitness_scores)
    best_scores.append(fitness_scores[best_idx])
    best_individuals.append(population[best_idx])

    # 2. Selection
    parents = roulette_selection(population, fitness_scores)

    # 3. Crossover
    children = single_point_crossover(parents)

    # 4. Mutation
    population = mutate(children)

In [7]:
best_scores
best_individuals

[array([55.02135296,  3.04242243, 52.47564316]),
 array([112.36203565,   9.50714306,  60.11150117]),
 array([112.36203565,   9.50714306,  60.11150117]),
 array([112.76874345,   9.50714306,  59.31422439]),
 array([104.73132728,   9.50714306,  59.31422439])]

## 6) Final result

In [8]:
best_index = np.argmax(best_scores)
best_solution = best_individuals[best_index]
best_fitness = best_scores[best_index]

print("Best solution:", best_solution)
print("Best fitness:", best_fitness)

Best solution: [104.73132728   9.50714306  59.31422439]
Best fitness: 33.586706876171895
